In [18]:
pip install requests beautifulsoup4 tldextract python-whois pytesseract pillow ipinfo fake_useragent

Note: you may need to restart the kernel to use updated packages.


In [3]:
import requests
from bs4 import BeautifulSoup
import tldextract
import csv
import os
from datetime import datetime
import socket
import ssl

# Konfigurasi
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
TIMEOUT = 10
KEYWORDS = ["judi", "slot", "casino", "taruhan", "poker", "deposit", "bonus", "bet", "sportsbook"]

def extract_features(url, label):
    try:
        # Ambil konten HTML
        response = requests.get(url, headers=HEADERS, timeout=TIMEOUT, verify=False)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Ekstrak domain dan subdomain
        ext = tldextract.extract(url)
        domain = ext.registered_domain
        subdomain = ext.subdomain
        domain_ext = ext.suffix
        
        # Ekstrak teks dan keyword
        text = soup.get_text(separator=' ', strip=True)
        keyword_count = sum(text.lower().count(keyword) for keyword in KEYWORDS)
        
        # Metadata HTML
        title = soup.title.string if soup.title else ""
        meta_desc = soup.find('meta', attrs={'name': 'description'})
        meta_desc = meta_desc['content'] if meta_desc else ""
        meta_keywords = soup.find('meta', attrs={'name': 'keywords'})
        meta_keywords = meta_keywords['content'] if meta_keywords else ""
        
        # Struktur halaman
        forms = len(soup.find_all('form'))
        external_links = [a['href'] for a in soup.find_all('a', href=True) if domain not in a['href']]
        
        # Gambar
        images = []
        for img in soup.find_all('img')[:5]:  # Ambil 5 gambar pertama
            img_src = img.get('src', '')
            img_alt = img.get('alt', '')
            images.append(f"{img_src} (Alt: {img_alt})")
        
        # SSL & Jaringan
        is_https = url.startswith('https://')
        ssl_valid = False
        if is_https:
            try:
                hostname = ext.domain
                ctx = ssl.create_default_context()
                with ctx.wrap_socket(socket.socket(), server_hostname=hostname) as s:
                    s.connect((hostname, 443))
                    ssl_valid = True
            except:
                ssl_valid = False
        
        # WHOIS (contoh sederhana)
        domain_age = "N/A"  # Implementasi lengkap butuh library whois
        
        # Data akhir
        return {
            'url': url,
            'domain': domain,
            'subdomain': subdomain,
            'domain_ext': domain_ext,
            'text_content': text[:500] + "...",  # Batasi teks
            'keyword_count': keyword_count,
            'meta_title': title,
            'meta_description': meta_desc,
            'meta_keywords': meta_keywords,
            'num_forms': forms,
            'external_links': ', '.join(external_links[:3]),  # Contoh 3 link
            'images': ' | '.join(images),
            'ssl_valid': ssl_valid,
            'is_https': is_https,
            'domain_age': domain_age,
            'is_gambling': label
        }
    
    except Exception as e:
        print(f"Error scraping {url}: {str(e)}")
        return None

def main():
    # Baca URL dari file
    with open('webjudi.txt', 'r') as f:
        judi_urls = [line.strip() for line in f.readlines()]
    
    with open('webnonjudi.txt', 'r') as f:
        non_judi_urls = [line.strip() for line in f.readlines()]
    
    # Scraping data
    data = []
    for url in judi_urls:
        features = extract_features(url, label=1)
        if features:
            data.append(features)
    
    for url in non_judi_urls:
        features = extract_features(url, label=0)
        if features:
            data.append(features)
    
    # Simpan ke CSV
    with open('gambling_dataset.csv', 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = [
            'url', 'domain', 'subdomain', 'domain_ext', 'text_content',
            'keyword_count', 'meta_title', 'meta_description', 'meta_keywords',
            'num_forms', 'external_links', 'images', 'ssl_valid', 'is_https',
            'domain_age', 'is_gambling'
        ]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(data)
    
    print(f"Dataset berhasil dibuat dengan {len(data)} entri!")

if __name__ == "__main__":
    main()

C:\Users\Raptor\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'pbsi.umk.ac.id'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\Raptor\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.detik.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Dataset berhasil dibuat dengan 2 entri!


In [13]:
import requests
from bs4 import BeautifulSoup
import tldextract
import csv
import time
import whois
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
# Konfigurasi
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
TIMEOUT = 10
KEYWORDS = ["judi", "slot", "casino", "taruhan", "poker", "deposit", "bonus", "bet", "sportsbook"]
DELAY = 2  # Delay antar-request (dalam detik)

def extract_features(url, label):
    try:
        # Delay antar-request
        time.sleep(DELAY)
        
        # Ambil konten HTML
        response = requests.get(url, headers=HEADERS, timeout=TIMEOUT, verify=False)
        response.raise_for_status()  # Raise exception jika status code bukan 200
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Ekstrak domain dan subdomain
        ext = tldextract.extract(url)
        domain = ext.registered_domain
        subdomain = ext.subdomain
        domain_ext = ext.suffix
        
        # Ekstrak teks dan keyword
        text = soup.get_text(separator=' ', strip=True)
        keyword_count = sum(text.lower().count(keyword) for keyword in KEYWORDS)
        
        # Metadata HTML
        title = soup.title.string if soup.title else ""
        meta_desc = soup.find('meta', attrs={'name': 'description'})
        meta_desc = meta_desc['content'] if meta_desc else ""
        meta_keywords = soup.find('meta', attrs={'name': 'keywords'})
        meta_keywords = meta_keywords['content'] if meta_keywords else ""
        
        # Struktur halaman
        forms = len(soup.find_all('form'))
        external_links = [a['href'] for a in soup.find_all('a', href=True) if domain not in a['href']]
        
        # Data akhir
        return {
            'url': url,
            'domain': domain,
            'subdomain': subdomain,
            'domain_ext': domain_ext,
            'text_content': text[:2000] + "...",  # Batasi teks
            'keyword_count': keyword_count,
            'meta_title': title,
            'meta_description': meta_desc,
            'meta_keywords': meta_keywords,
            'num_forms': forms,
            'external_links': ', '.join(external_links[:3]),  # Contoh 3 link
            'is_gambling': label
        }
    
    except requests.exceptions.RequestException as e:
        print(f"Error accessing {url}: {str(e)}")
        return None
    except Exception as e:
        print(f"Unexpected error with {url}: {str(e)}")
        return None

def main():
    # Baca URL dari file
    with open('webjudi.txt', 'r') as f:
        judi_urls = [line.strip() for line in f.readlines()]
    
    with open('webnonjudi.txt', 'r') as f:
        non_judi_urls = [line.strip() for line in f.readlines()]
    
    # Scraping data
    data = []
    for url in judi_urls:
        features = extract_features(url, label=1)
        if features:
            data.append(features)
    
    for url in non_judi_urls:
        features = extract_features(url, label=0)
        if features:
            data.append(features)
    
    # Simpan ke CSV
    with open('gambling_dataset.csv', 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = [
            'url', 'domain', 'subdomain', 'domain_ext', 'text_content',
            'keyword_count', 'meta_title', 'meta_description', 'meta_keywords',
            'num_forms', 'external_links', 'is_gambling'
        ]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(data)
    
    print(f"Dataset berhasil dibuat dengan {len(data)} entri!")

if __name__ == "__main__":
    main()

Dataset berhasil dibuat dengan 2 entri!


In [14]:
import requests
from bs4 import BeautifulSoup

def scrape_bing():
    query = "situs+judi+online"
    urls = []
    for page in range(1, 6):  # Ambil 5 halaman
        url = f"https://www.bing.com/search?q={query}&first={page * 10}"
        response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
        soup = BeautifulSoup(response.text, "html.parser")
        for link in soup.find_all("a", href=True):
            href = link["href"]
            if href.startswith("http") and "bing.com" not in href:
                urls.append(href)
    return urls

gambling_urls = scrape_bing()
print("Daftar URL:", gambling_urls)

Daftar URL: ['https://www.ugdewapot.com/', 'https://www.ugdewapot.com/', 'https://katadata.co.id/digital/teknologi/66a89a6a0149b/daftar-10-aplikasi-judi-online-paling-sering-dipakai-di-indonesia', 'https://katadata.co.id/digital/teknologi/66a89a6a0149b/daftar-10-aplikasi-judi-online-paling-sering-dipakai-di-indonesia', 'https://news.republika.co.id/berita/smff5x409/tak-seperti-dulu-daftar-situs-judi-online-yang-diblokir-kemkomdigi-akan-diupdate-harian', 'https://news.republika.co.id/berita/smff5x409/tak-seperti-dulu-daftar-situs-judi-online-yang-diblokir-kemkomdigi-akan-diupdate-harian', 'https://nasional.kompas.com/read/2025/01/20/22193961/situs-judol-yang-diblokir-muncul-lagi-pakai-nama-sama-cuma-beda-url', 'https://nasional.kompas.com/read/2025/01/20/22193961/situs-judol-yang-diblokir-muncul-lagi-pakai-nama-sama-cuma-beda-url', 'https://megapolitan.kompas.com/read/2024/11/07/06000091/pesan-dari-pembuat-situs-judi-online---jangan-percaya-dengan-judi-online-', 'https://megapolitan.kom

In [15]:
import requests
from bs4 import BeautifulSoup
import time
from urllib.parse import urlparse

def scrape_bing():
    query = "situs+judi+online"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept-Language': 'en-US,en;q=0.9'
    }
    
    seen_urls = set()
    base_domain = 'bing.com'
    
    for page in range(0, 50, 10):  # 5 halaman (0,10,20,30,40)
        try:
            url = f"https://www.bing.com/search?q={query}&first={page+1}"
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            
            soup = BeautifulSoup(response.text, "html.parser")
            
            # Target hanya hasil pencarian utama
            results = soup.find_all('li', class_='b_algo')
            
            for result in results:
                link = result.find('a')
                if link and link['href']:
                    parsed = urlparse(link['href'])
                    # Filter out unwanted URLs
                    if (
                        parsed.netloc != base_domain and
                        not parsed.netloc.endswith('.' + base_domain) and
                        'webcache.googleusercontent.com' not in parsed.netloc
                    ):
                        final_url = parsed.geturl()
                        if final_url not in seen_urls:
                            seen_urls.add(final_url)
            
            time.sleep(3)  # Delay penting untuk hindari blokir
            
        except Exception as e:
            print(f"Error on page {page//10 + 1}: {str(e)}")
            continue
            
    return list(seen_urls)

gambling_urls = scrape_bing()
print(f"Found {len(gambling_urls)} unique URLs:")
for idx, url in enumerate(gambling_urls[:20], 1):  # Tampilkan 20 pertama
    print(f"{idx}. {url}")

Found 10 unique URLs:
1. https://www.casino.org/id-id/
2. https://katadata.co.id/digital/teknologi/66a89a6a0149b/daftar-10-aplikasi-judi-online-paling-sering-dipakai-di-indonesia
3. https://www.188bet.com/id-id
4. https://tripkuy.co.id/blog/5-situs-judi-online-dengan-server-terbaik/
5. https://megapolitan.kompas.com/read/2024/11/07/06000091/pesan-dari-pembuat-situs-judi-online---jangan-percaya-dengan-judi-online-
6. https://www.tempo.co/hukum/polri-bongkar-3-situs-judi-online-sebut-sulit-diberantas-walau-telah-diblokir-1198048
7. https://nasional.kompas.com/read/2025/01/20/22193961/situs-judol-yang-diblokir-muncul-lagi-pakai-nama-sama-cuma-beda-url
8. https://news.republika.co.id/berita/smff5x409/tak-seperti-dulu-daftar-situs-judi-online-yang-diblokir-kemkomdigi-akan-diupdate-harian
9. https://www.ugdewapot.com/
10. https://inet.detik.com/law-and-policy/d-7402719/ini-dia-deretan-situs-judi-online-yang-diblokir-kominfo


In [16]:
import requests
from bs4 import BeautifulSoup
import time
import random
from urllib.parse import urlparse

def scrape_bing():
    query = "situs+judi+online"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept-Language': 'en-US,en;q=0.9',
        'Referer': 'https://www.bing.com/'
    }
    
    seen_urls = set()
    session = requests.Session()  # Gunakan session untuk pertahankan cookie
    
    for page in range(0, 5):  # 5 halaman
        try:
            # Parameter "first" harus mulai dari 1, 11, 21, dst.
            url = f"https://www.bing.com/search?q={query}&first={page * 10 + 1}"
            response = session.get(url, headers=headers, timeout=15)
            response.raise_for_status()
            
            soup = BeautifulSoup(response.text, "html.parser")
            
            # Cari semua elemen <a> di dalam tag <li> dengan class "b_algo" (update jika perlu)
            results = soup.select('li.b_algo h2 a')
            
            for link in results:
                href = link.get('href')
                if href:
                    parsed = urlparse(href)
                    # Filter URL Bing dan cache
                    if "bing.com" not in parsed.netloc and "webcache" not in parsed.netloc:
                        final_url = parsed.geturl()
                        seen_urls.add(final_url)
            
            # Delay acak antara 2-5 detik
            time.sleep(random.uniform(2, 5))
            
        except Exception as e:
            print(f"Error pada halaman {page + 1}: {str(e)}")
            continue
            
    return list(seen_urls)

gambling_urls = scrape_bing()
print(f"Jumlah URL unik: {len(gambling_urls)}")
for url in gambling_urls:
    print(url)

Jumlah URL unik: 10
https://www.casino.org/id-id/
https://katadata.co.id/digital/teknologi/66a89a6a0149b/daftar-10-aplikasi-judi-online-paling-sering-dipakai-di-indonesia
https://www.188bet.com/id-id
https://tripkuy.co.id/blog/5-situs-judi-online-dengan-server-terbaik/
https://megapolitan.kompas.com/read/2024/11/07/06000091/pesan-dari-pembuat-situs-judi-online---jangan-percaya-dengan-judi-online-
https://www.tempo.co/hukum/polri-bongkar-3-situs-judi-online-sebut-sulit-diberantas-walau-telah-diblokir-1198048
https://nasional.kompas.com/read/2025/01/20/22193961/situs-judol-yang-diblokir-muncul-lagi-pakai-nama-sama-cuma-beda-url
https://news.republika.co.id/berita/smff5x409/tak-seperti-dulu-daftar-situs-judi-online-yang-diblokir-kemkomdigi-akan-diupdate-harian
https://www.ugdewapot.com/
https://inet.detik.com/law-and-policy/d-7402719/ini-dia-deretan-situs-judi-online-yang-diblokir-kominfo


In [19]:
import requests
from bs4 import BeautifulSoup
import time
import random
from urllib.parse import urlparse
from fake_useragent import UserAgent

def scrape_bing():
    ua = UserAgent()
    query = "situs+judi+online"
    seen_urls = set()
    
    for page in range(0, 5):  # 5 halaman
        try:
            # Generate random headers tiap request
            headers = {
                'User-Agent': ua.random,
                'Accept-Language': 'en-US,en;q=0.9',
                'Referer': 'https://www.bing.com/' if page == 0 else f'https://www.bing.com/search?q={query}&first={page*10}'
            }
            
            url = f"https://www.bing.com/search?q={query}&first={page*10+1}"
            
            response = requests.get(url, headers=headers, timeout=15)
            print(f"\n=== Response Halaman {page+1} ===")
            print(f"Status Code: {response.status_code}")
            print(f"Headers: {response.headers}\n")
            
            # Simpan response ke file untuk inspeksi
            with open(f"bing_page_{page+1}.html", "w", encoding="utf-8") as f:
                f.write(response.text)
            
            soup = BeautifulSoup(response.text, "html.parser")
            
            # Debug: Cek apakah ada element hasil pencarian
            results = soup.find_all(['li', 'div'], class_=['b_algo', 'b_result'])
            print(f"Jumlah Element Hasil Ditemukan: {len(results)}")
            
            # Ekstrak URL dengan selector alternatif
            for result in results:
                link = result.find('a', href=True)
                if link:
                    href = link['href']
                    parsed = urlparse(href)
                    if "bing.com" not in parsed.netloc:
                        print(f"URL Mentah: {href}")
                        seen_urls.add(href.split('?')[0])  # Hapus parameter tracking
            
            # Delay acak lebih natural
            time.sleep(random.uniform(3, 8))
            
        except Exception as e:
            print(f"Error: {str(e)}")
            continue
            
    return list(seen_urls)

# Jalankan dan tampilkan hasil
gambling_urls = scrape_bing()
print(f"\nTotal URL Unik: {len(gambling_urls)}")
print("Daftar URL:")
for idx, url in enumerate(gambling_urls, 1):
    print(f"{idx}. {url}")


=== Response Halaman 1 ===
Status Code: 200
Headers: {'Content-Type': 'text/html; charset=utf-8', 'Cache-Control': 'private, max-age=0', 'Content-Encoding': 'gzip', 'Expires': 'Thu, 23 Jan 2025 16:41:24 GMT', 'Vary': 'Accept-Encoding', 'X-EventID': '6792717089cc407bb9d0abf5289ded58', 'UserAgentReductionOptOut': 'A7kgTC5xdZ2WIVGZEfb1hUoNuvjzOZX3VIV/BA6C18kQOOF50Q0D3oWoAm49k3BQImkujKILc7JmPysWk3CSjwUAAACMeyJvcmlnaW4iOiJodHRwczovL3d3dy5iaW5nLmNvbTo0NDMiLCJmZWF0dXJlIjoiU2VuZEZ1bGxVc2VyQWdlbnRBZnRlclJlZHVjdGlvbiIsImV4cGlyeSI6MTY4NDg4NjM5OSwiaXNTdWJkb21haW4iOnRydWUsImlzVGhpcmRQYXJ0eSI6dHJ1ZX0=', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; preload', 'Content-Security-Policy': "script-src https: 'strict-dynamic' 'report-sample' 'wasm-unsafe-eval' 'nonce-DG+YoRF9dAnwMefgVV3IhOAWEg6fSfK3Dk2SaSKGpDY='; base-uri 'self';report-to csp-endpoint", 'Report-To': '{"group":"csp-endpoint","max_age":86400,"endpoints":[{"url":"https://aefd.nelreports.net/api/report?cat=bingcsp"}]}', 

In [34]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time

options = Options()
options.add_argument("--headless")  # Jalankan di background
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36")
service = Service('C:\Program Files\chromedriver\chromedriver.exe')  # Ganti dengan path manual
driver = webdriver.Chrome(service=service, options=options)

try:
    for page in range(0, 5):
        url = f"https://www.bing.com/search?q=situs+judi+online&first={page*10+1}"
        driver.get(url)
        time.sleep(5)  # Tunggu halaman termuat
        
        # Ambil semua link hasil pencarian
        results = driver.find_elements(By.CSS_SELECTOR, 'li.b_algo h2 a')
        for link in results:
            print(link.get_attribute("href"))
            
        time.sleep(random.uniform(5, 10))
finally:
    driver.quit()

<>:10: SyntaxWarning: invalid escape sequence '\P'
<>:10: SyntaxWarning: invalid escape sequence '\P'
C:\Users\Raptor\AppData\Local\Temp\ipykernel_22268\2215342993.py:10: SyntaxWarning: invalid escape sequence '\P'
  service = Service('C:\Program Files\chromedriver\chromedriver.exe')  # Ganti dengan path manual
C:\Users\Raptor\AppData\Local\Temp\ipykernel_22268\2215342993.py:10: SyntaxWarning: invalid escape sequence '\P'
  service = Service('C:\Program Files\chromedriver\chromedriver.exe')  # Ganti dengan path manual


TypeError: WebDriver.__init__() got an unexpected keyword argument 'service'

In [23]:
pip install --upgrade webdriver-manager selenium

  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
   ---------------------------------------- 0.0/9.5 MB ? eta -:--:--
   --------- ------------------------------ 2.4/9.5 MB 12.2 MB/s eta 0:00:01
   ------------------- -------------------- 4.7/9.5 MB 11.9 MB/s eta 0:00:01
   ----------------------------- ---------- 7.1/9.5 MB 11.8 MB/s eta 0:00:01
   ---------------------------------------  9.4/9.5 MB 12.0 MB/s eta 0:00:01
   ---------------------------------------- 9.5/9.5 MB 11.2 MB/s eta 0:00:00
Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
  Attempting uninstall: websocket-client
    Found existing installation: websocket-client 1.7.0
    Uninstalling websocket-client-1.7.0:
      Successfully uninstalled websocket-client-1.7.0
  Attempting uninstall: webdriver-manager
    Found existing installation: webdriver-manager 3.7.0
    Uninstalling webdriver-manager-3.7.0:
      Successfully uninstalled webdriver-manager-3.7.0
  Attempting uninsta

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
seleniumbase 4.22.3 requires selenium==4.16.0; python_version >= "3.8", but you have selenium 4.28.1 which is incompatible.
udemy-enroller 4.1.3 requires selenium<4.0.0,>=3.141.0, but you have selenium 4.28.1 which is incompatible.
udemy-enroller 4.1.3 requires webdriver-manager<4.0.0,>=3.7.0, but you have webdriver-manager 4.0.2 which is incompatible.


In [32]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time
import random

options = Options()
options.add_argument("--headless")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36")

# Ambil major version dari Chrome (131)
chrome_version = '131'  # Ganti dengan major version Chrome Anda

# Gunakan versi ChromeDriver yang sesuai
service = Service(ChromeDriverManager(version=f"132.0.6834.110").install())
driver = webdriver.Chrome(service=service, options=options)

try:
    for page in range(0, 5):
        url = f"https://www.bing.com/search?q=situs+judi+online&cc=ID&first={page*10+1}"
        driver.get(url)
        time.sleep(5)
        
        results = driver.find_elements(By.CSS_SELECTOR, 'li.b_algo h2 a')
        unique_urls = set()
        
        for link in results:
            url = link.get_attribute("href")
            if url and "bing.com" not in url:
                unique_urls.add(url)
        
        print(f"Halaman {page+1}: {len(unique_urls)} URL ditemukan")
        for idx, url in enumerate(unique_urls, 1):
            print(f"{idx}. {url}")
            
        time.sleep(random.uniform(5, 10))
        
finally:
    driver.quit()

[WDM] - ====== WebDriver manager ======
[WDM] - There is no [win32] chromedriver for browser  in cache
[WDM] - About to download new driver from https://chromedriver.storage.googleapis.com/132.0.6834.110/chromedriver_win32.zip


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\Raptor\AppData\Local\Programs\Python\Python312\Lib\site-packages\IPython\core\interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Raptor\AppData\Local\Temp\ipykernel_22268\1691210113.py", line 17, in <module>
    service = Service(ChromeDriverManager(version=f"132.0.6834.110").install())
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Raptor\AppData\Local\Programs\Python\Python312\Lib\site-packages\webdriver_manager\chrome.py", line 37, in install
    )
      
  File "C:\Users\Raptor\AppData\Local\Programs\Python\Python312\Lib\site-packages\webdriver_manager\core\manager.py", line 33, in _get_driver_path
  File "C:\Users\Raptor\AppData\Local\Programs\Python\Python312\Lib\site-packages\webdriver_manager\core\download_manager.py", line 28, in download_file
    log(f"About to download new driver from {url}")
               ^^^^

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import random

options = Options()
options.add_argument("--headless")  # Run in background
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36")

# Initialize the Chrome driver without service argument
driver = webdriver.Chrome(options=options)

try:
    for page in range(0, 5):
        url = f"https://www.bing.com/search?q=situs+judi+online&first={page*10+1}"
        driver.get(url)
        time.sleep(5)  # Wait for the page to load
        
        # Get all the search result links
        results = driver.find_elements(By.CSS_SELECTOR, 'li.b_algo h2 a')
        for link in results:
            print(link.get_attribute("href"))
            
        time.sleep(random.uniform(5, 10))
finally:
    driver.quit()

https://www.newsbtc.com/online-slots-indonesia/
https://katadata.co.id/digital/teknologi/66a89a6a0149b/daftar-10-aplikasi-judi-online-paling-sering-dipakai-di-indonesia
https://www.sbobet.com/
https://www.casino.org/id-id/
https://nasional.kompas.com/read/2025/01/20/22193961/situs-judol-yang-diblokir-muncul-lagi-pakai-nama-sama-cuma-beda-url
https://bp77asian.com/id_id/
https://www.188bet.com/id-id
https://megapolitan.kompas.com/read/2024/11/07/06000091/pesan-dari-pembuat-situs-judi-online---jangan-percaya-dengan-judi-online-
https://www.tempo.co/hukum/polri-bongkar-3-situs-judi-online-sebut-sulit-diberantas-walau-telah-diblokir-1198048
https://shiobet.com/
https://shiobet.com/
https://bk8.plus/
https://www.tempo.co/ekonomi/ini-kronologi-dan-modus-pejabat-kementerian-komdigi-menjaga-situs-judi-online-1162708
https://istanacasino.com/slot-online
https://nasional.kompas.com/read/2024/11/06/18035331/tutup-220000-situs-judi-online-menkomdigi-ini-bukan-prestasi-kita-akan-kejar
https://www.b

In [35]:
pip show selenium

Name: selenium
Version: 4.28.1
Summary: Official Python bindings for Selenium WebDriver
Home-page: https://www.selenium.dev
Author: 
Author-email: 
License: Apache 2.0
Location: C:\Users\Raptor\AppData\Local\Programs\Python\Python312\Lib\site-packages
Requires: certifi, trio, trio-websocket, typing_extensions, urllib3, websocket-client
Required-by: seleniumbase, udemy-enroller
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import random

# Baca kata kunci dari file
with open('keywords.txt', 'r') as f:
    keywords = [line.strip() for line in f.readlines() if line.strip()]

# Setup Chrome options
options = Options()
options.add_argument("--headless")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36")
options.add_argument("--disable-blink-features=AutomationControlled")
driver = webdriver.Chrome(options=options)
collected_urls = []  # Untuk menyimpan semua URL

try:
    for keyword in keywords:
        print(f"Memproses kata kunci: {keyword}")
        
        for page in range(0, 5):
            url = f"https://www.bing.com/search?q={keyword.replace(' ', '+')}&first={page*10+1}"
            driver.get(url)
            time.sleep(3)
            
            # Ambil hasil pencarian
            results = driver.find_elements(By.CSS_SELECTOR, 'li.b_algo h2 a')
            
            for link in results:
                url = link.get_attribute("href")
                if url and "bing.com" not in url:
                    collected_urls.append(url)
            
            time.sleep(random.uniform(2, 5))

finally:
    driver.quit()
    # Simpan hasil ke file
    with open('results.txt', 'w') as f:
        for url in list(set(collected_urls)):  # Hapus duplikat dengan set()
            f.write(url + '\n')
            
print(f"Total {len(collected_urls)} URL unik tersimpan di results.txt")

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import random

# Baca kata kunci dari file
with open('keywords.txt', 'r') as f:
    keywords = [line.strip() for line in f.readlines() if line.strip()]

# Setup Chrome
options = Options()
options.add_argument("--headless")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36")
options.add_argument("--disable-blink-features=AutomationControlled")
driver = webdriver.Chrome(options=options)
collected_urls = set()  # Gunakan set() untuk otomatis hapus duplikat

try:
    # Buka file dalam mode append ('a')
    with open('results.txt', 'a') as f:  
        for keyword in keywords:
            print(f"Memproses kata kunci: {keyword}")
            
            for page in range(0, 5):
                url = f"https://www.bing.com/search?q={keyword.replace(' ', '+')}&first={page*10+1}"
                driver.get(url)
                time.sleep(3)
                
                results = driver.find_elements(By.CSS_SELECTOR, 'li.b_algo h2 a')
                
                # Simpan langsung ke file (append)
                for link in results:
                    url = link.get_attribute("href")
                    if url and "bing.com" not in url and url not in collected_urls:
                        f.write(url + '\n')
                        collected_urls.add(url)  # Catat URL yang sudah disimpan
                
                time.sleep(random.uniform(2, 5))

finally:
    driver.quit()
    print(f"Total {len(collected_urls)} URL unik tersimpan di results.txt")

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import random
import csv

# Baca kata kunci dari file
with open('keywords.txt', 'r') as f:
    keywords = [line.strip() for line in f.readlines() if line.strip()]

# Setup Chrome
options = Options()
options.add_argument("--headless")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36")

driver = webdriver.Chrome(options=options)
collected_data = set()  # Simpan dalam format (keyword, url)

try:
    with open('results.csv', 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['keyword', 'url'])  # Header
        
        for keyword in keywords:
            print(f"Memproses kata kunci: {keyword}")
            
            for page in range(0, 5):
                url = f"https://www.bing.com/search?q={keyword.replace(' ', '+')}&first={page*10+1}"
                driver.get(url)
                time.sleep(3)
                
                results = driver.find_elements(By.CSS_SELECTOR, 'li.b_algo h2 a')
                
                for link in results:
                    url = link.get_attribute("href")
                    if url and "bing.com" not in url:
                        # Cek duplikat (keyword, url)
                        if (keyword, url) not in collected_data:
                            writer.writerow([keyword, url])
                            collected_data.add((keyword, url))  # Simpan pasangan unik
                
                time.sleep(random.uniform(2, 5))

finally:
    driver.quit()
    print(f"Total {len(collected_data)} entri tersimpan di results.csv")

Memproses kata kunci: judi online terpercaya
Memproses kata kunci: situs judi resmi
Total 91 entri tersimpan di results.csv


In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import random
import csv
import os

# Baca kata kunci dari file
with open('keywords.txt', 'r') as f:
    keywords = [line.strip() for line in f.readlines() if line.strip()]

# Setup Chrome
options = Options()
options.add_argument("--headless")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36")

driver = webdriver.Chrome(options=options)
collected_data = set()  # Simpan kombinasi unik (keyword, url)

# Cek apakah file CSV sudah ada
file_exists = os.path.isfile('results.csv')

try:
    with open('results.csv', 'a', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        
        # Tulis header hanya jika file baru
        if not file_exists:
            writer.writerow(['keyword', 'url'])
        
        for keyword in keywords:
            keyword_urls = []  # Kumpulkan URL per keyword
            
            print(f"Memproses kata kunci: {keyword}")
            
            for page in range(0, 5):
                url = f"https://www.bing.com/search?q={keyword.replace(' ', '+')}&first={page*10+1}"
                driver.get(url)
                time.sleep(3)
                
                results = driver.find_elements(By.CSS_SELECTOR, 'li.b_algo h2 a')
                
                for link in results:
                    url = link.get_attribute("href")
                    if url and "bing.com" not in url:
                        # Cek duplikat global
                        if (keyword, url) not in collected_data:
                            keyword_urls.append(url)
                            collected_data.add((keyword, url))
                
                time.sleep(random.uniform(2, 5))
            
            # Tulis hasil per keyword ke CSV
            for url in keyword_urls:
                writer.writerow([keyword, url])
            
            print(f"✅ {len(keyword_urls)} URL disimpan untuk '{keyword}'")

finally:
    driver.quit()
    print(f"\nTotal {len(collected_data)} entri unik tersimpan di results.csv")

Memproses kata kunci: agen judi slot
✅ 43 URL disimpan untuk 'agen judi slot'
Memproses kata kunci: judi deposit pulsa
✅ 47 URL disimpan untuk 'judi deposit pulsa'

Total 90 entri unik tersimpan di results.csv


In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import random
import csv
import os

# Baca kata kunci dari file
with open('keywords.txt', 'r') as f:
    keywords = [line.strip() for line in f.readlines() if line.strip()]

# Setup Chrome
options = Options()
options.add_argument("--headless")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36")

driver = webdriver.Chrome(options=options)
collected_data = set()  # Untuk cek duplikat global

# Cek apakah file CSV sudah ada
file_exists = os.path.isfile('results.csv')

try:
    # Buka file CSV sekali saja (mode append)
    with open('results.csv', 'a', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        
        # Tulis header jika file baru
        if not file_exists:
            writer.writerow(['keyword', 'url'])
            csvfile.flush()  # Pastikan header langsung tersimpan
        
        for keyword in keywords:
            keyword_urls = []  # Kumpulkan URL per keyword
            
            print(f"Memproses kata kunci: {keyword}")
            
            for page in range(0, 5):
                url = f"https://www.bing.com/search?q={keyword.replace(' ', '+')}&first={page*10+1}"
                driver.get(url)
                time.sleep(3)
                
                results = driver.find_elements(By.CSS_SELECTOR, 'li.b_algo h2 a')
                
                for link in results:
                    url = link.get_attribute("href")
                    if url and "bing.com" not in url:
                        # Cek duplikat global
                        if (keyword, url) not in collected_data:
                            keyword_urls.append(url)
                            collected_data.add((keyword, url))
                
                time.sleep(random.uniform(2, 5))
            
            # Tulis hasil per keyword ke CSV dan langsung simpan
            for url in keyword_urls:
                writer.writerow([keyword, url])
            
            # Force flush data ke file
            csvfile.flush()
            os.fsync(csvfile.fileno())  # Pastikan data disimpan ke disk
            
            print(f"✅ {len(keyword_urls)} URL disimpan untuk '{keyword}'")

finally:
    driver.quit()
    print(f"\nTotal {len(collected_data)} entri unik tersimpan di results.csv")

Memproses kata kunci: judi online terpercaya
✅ 46 URL disimpan untuk 'judi online terpercaya'
Memproses kata kunci: situs judi resmi
✅ 45 URL disimpan untuk 'situs judi resmi'
Memproses kata kunci: agen judi slot
✅ 41 URL disimpan untuk 'agen judi slot'
Memproses kata kunci: judi deposit pulsa
✅ 42 URL disimpan untuk 'judi deposit pulsa'
Memproses kata kunci: judi togel online
✅ 43 URL disimpan untuk 'judi togel online'
Memproses kata kunci: judi tembak ikan
✅ 44 URL disimpan untuk 'judi tembak ikan'
Memproses kata kunci: cara daftar judi online
✅ 45 URL disimpan untuk 'cara daftar judi online'
Memproses kata kunci: judi online tanpa deposit
✅ 47 URL disimpan untuk 'judi online tanpa deposit'
Memproses kata kunci: judi ilegal terbesar
✅ 49 URL disimpan untuk 'judi ilegal terbesar'
Memproses kata kunci: judi online 24 jam
✅ 42 URL disimpan untuk 'judi online 24 jam'
Memproses kata kunci: slot gacor hari ini
✅ 46 URL disimpan untuk 'slot gacor hari ini'
Memproses kata kunci: slot online 


KeyboardInterrupt



In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import threading
import time
import random
import csv
import os
from queue import Queue

# Konfigurasi
MAX_THREADS = 10
keywords_queue = Queue()
lock = threading.Lock()
collected_data = set()

# Baca kata kunci dan masukkan ke queue
with open('keywords.txt', 'r') as f:
    keywords = [line.strip() for line in f.readlines() if line.strip()]
    for kw in keywords:
        keywords_queue.put(kw)

def worker():
    # Deklarasi global untuk file_exists
    global file_exists
    
    options = Options()
    options.add_argument("--headless")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36")
    
    driver = webdriver.Chrome(options=options)
    
    while not keywords_queue.empty():
        keyword = keywords_queue.get()
        
        try:
            keyword_urls = []
            
            for page in range(0, 5):
                url = f"https://www.bing.com/search?q={keyword.replace(' ', '+')}&first={page*10+1}"
                driver.get(url)
                time.sleep(3)
                
                results = driver.find_elements(By.CSS_SELECTOR, 'li.b_algo h2 a')
                
                for link in results:
                    url = link.get_attribute("href")
                    if url and "bing.com" not in url:
                        with lock:
                            if (keyword, url) not in collected_data:
                                keyword_urls.append(url)
                                collected_data.add((keyword, url))
                
                time.sleep(random.uniform(2, 5))
            
            with lock:
                # Cek ulang file_exists di dalam lock
                file_exists = os.path.isfile('results_ac.id.csv')
                
                with open('results_ac.id.csv', 'a', newline='', encoding='utf-8') as csvfile:
                    writer = csv.writer(csvfile)
                    if not file_exists:
                        writer.writerow(['keyword', 'url'])
                        file_exists = True  # Update status
                    for url in keyword_urls:
                        writer.writerow([keyword, url])
                
                print(f"✅ {len(keyword_urls)} URL disimpan untuk '{keyword}'")
        
        finally:
            keywords_queue.task_done()

    driver.quit()

# Inisialisasi file_exists di luar thread
file_exists = os.path.isfile('results_ac.id.csv')

# Jalankan thread
threads = []
for _ in range(MAX_THREADS):
    t = threading.Thread(target=worker)
    t.start()
    threads.append(t)

for t in threads:
    t.join()

print(f"\nTotal {len(collected_data)} entri unik tersimpan di results_ac.id.csv")

✅ 33 URL disimpan untuk 'judi online terpercaya site%3A.ac.id'
✅ 48 URL disimpan untuk 'agen judi slot site%3A.ac.id'
✅ 32 URL disimpan untuk 'judi online tanpa deposit site%3A.ac.id'
✅ 43 URL disimpan untuk 'judi togel online site%3A.ac.id'
✅ 40 URL disimpan untuk 'situs judi resmi site%3A.ac.id'
✅ 46 URL disimpan untuk 'judi deposit pulsa site%3A.ac.id'
✅ 48 URL disimpan untuk 'cara daftar judi online site%3A.ac.id'
✅ 44 URL disimpan untuk 'judi ilegal terbesar site%3A.ac.id'
✅ 44 URL disimpan untuk 'judi tembak ikan site%3A.ac.id'
✅ 40 URL disimpan untuk 'judi online 24 jam site%3A.ac.id'
✅ 41 URL disimpan untuk 'slot online terbaik site%3A.ac.id'
✅ 45 URL disimpan untuk 'slot gacor hari ini site%3A.ac.id'
✅ 40 URL disimpan untuk 'situs slot maxwin site%3A.ac.id'
✅ 47 URL disimpan untuk 'slot deposit 10 ribu site%3A.ac.id'
✅ 43 URL disimpan untuk 'roulette online uang asli site%3A.ac.id'
✅ 47 URL disimpan untuk 'demo slot pragmatic site%3A.ac.id'
✅ 34 URL disimpan untuk 'casino onli

Exception in thread Thread-22 (worker):
Traceback (most recent call last):
  File "C:\Users\Raptor\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1052, in _bootstrap_inner
    self.run()
  File "C:\Users\Raptor\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 989, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Raptor\AppData\Local\Temp\ipykernel_24240\674919173.py", line 47, in worker
  File "C:\Users\Raptor\AppData\Local\Programs\Python\Python312\Lib\site-packages\selenium\webdriver\remote\webelement.py", line 231, in get_attribute
    attribute_value = self.parent.execute_script(
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Raptor\AppData\Local\Programs\Python\Python312\Lib\site-packages\selenium\webdriver\remote\webdriver.py", line 528, in execute_script
    return self.execute(command, {"script": script, "args": converted_args})["value"]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

✅ 44 URL disimpan untuk 'domino qiu qiu deposit murah site%3A.ac.id'
✅ 42 URL disimpan untuk 'domino qiu qiu cynking versi lama site%3A.ac.id'
✅ 44 URL disimpan untuk 'domino qiu qiu depposit site%3A.ac.id'
✅ 44 URL disimpan untuk 'domino qiu qiu domino 99 apk site%3A.ac.id'
✅ 47 URL disimpan untuk 'domino qiu qiu domino card site%3A.ac.id'
✅ 45 URL disimpan untuk 'domino qiu qiu download for pc site%3A.ac.id'
✅ 43 URL disimpan untuk 'domino qiu qiu fcebook site%3A.ac.id'
✅ 42 URL disimpan untuk 'domino qiu qiu for cashtree mod apk site%3A.ac.id'
✅ 43 URL disimpan untuk 'domino qiu qiu game for c site%3A.ac.id'
✅ 45 URL disimpan untuk 'domino qiu qiu games site%3A.ac.id'
✅ 41 URL disimpan untuk 'domino qiu qiu gaple apk site%3A.ac.id'
✅ 43 URL disimpan untuk 'domino qiu qiu gaple texas lokal online site%3A.ac.id'
✅ 47 URL disimpan untuk 'domino qiu qiu google play site%3A.ac.id'
✅ 43 URL disimpan untuk 'domino qiu qiu id site%3A.ac.id'
✅ 44 URL disimpan untuk 'domino qiu qiu indoplay a